<a href="https://colab.research.google.com/github/olivermtoth/CPROG_FINAL/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import dependencies

In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import pathlib

### Import data

In [5]:
data_csv =  sorted(str(p) for p in pathlib.Path('data').glob("*.csv"))

In [6]:
len(data_csv)

39

In [17]:
TrainX = []
TrainY = []
TestX = []
TestY = []
ValidX = []
ValidY = []

In [18]:
period = 20 # days
#TODO: Nomralization
for file in data_csv:
    ds = pd.read_csv(file)
    ds = ds.iloc[:, 1:8]
    num_row = len(ds)
    # Normailize the dat
    for col in ds.columns:
      x_min = ds[col].min()
      x_max = ds[col].max()
      ds[col] = (ds[col] - x_min) / (x_max - x_min)
    num_train = int(num_row*0.70) - 20
    num_valid = int((num_row - num_train)/3)
    num_test = num_row - (num_train + num_valid)
    Y = ds['price']
    X = ds.iloc[:, 0:6]
    for i in range(num_row-20):
        if i < num_train:
            TrainY.append(Y.iloc[i+period])
            TrainX.append(X.iloc[i:i+period].to_numpy())
        elif i < num_train + num_test:
            ValidY.append(Y.iloc[i+period])
            ValidX.append(X.iloc[i:i+period].to_numpy())
        else:
            TestY.append(Y.iloc[i+period])
            TestX.append(X.iloc[i:i+period].to_numpy())
    break

#### Get the size of test dataset and confirming correct datatype

In [9]:
print(len(TrainX)+len(ValidX)+len(TestX))
print(TrainX[0].shape)
print(type(TrainX[0]))

5876
(20, 6)
<class 'numpy.ndarray'>


Fun fact: We have more than 200,000 thousand days of data, which is about 548.5 years!!!

#### Converting arrays to Numpy array for the faster computation.

In [19]:
TrainX = np.array(TrainX)
TrainY = np.array(TrainY)
TestX = np.array(TestX)
TestY = np.array(TestY)
ValidX = np.array(ValidX)
ValidY = np.array(ValidY)

#### Save datasets as csv for future use

In [ ]:
#TrainX.tofile('./data/TrainX.csv', sep=',')
#TrainY.tofile('./data/TrainY.csv', sep=',')
#TestX.tofile('./data/TestX.csv', sep=',')
#TestY.tofile('./data/TestY.csv', sep=',')
#ValidX.tofile('./data/ValidX.csv', sep=',')
#ValidY.tofile('./data/ValidY.csv', sep=',')

#### Make model

In [ ]:
class SoftAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(SoftAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [37]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(120, dropout=0.20, return_sequences=True, activation ='tanh'),
    # SoftAttention(60),
    tf.keras.layers.Dense(60, activation='tanh'),
    tf.keras.layers.Dense(1)
])

In [39]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=["mae"])

In [40]:
history = model.fit(x = TrainX, y = TrainY, batch_size=10, epochs=100, validation_data=(ValidX, ValidY))

Epoch 1/100
411/411 [==============================] - 6s 8ms/step - loss: 0.0024 - mae: 0.0399 - val_loss: 0.0703 - val_mae: 0.2222
Epoch 2/100
411/411 [==============================] - 3s 8ms/step - loss: 0.0023 - mae: 0.0399 - val_loss: 0.0764 - val_mae: 0.2361
Epoch 3/100
411/411 [==============================] - 3s 8ms/step - loss: 0.0023 - mae: 0.0399 - val_loss: 0.0660 - val_mae: 0.2147
Epoch 4/100
411/411 [==============================] - 3s 7ms/step - loss: 0.0022 - mae: 0.0397 - val_loss: 0.0725 - val_mae: 0.2291
Epoch 5/100
411/411 [==============================] - 3s 8ms/step - loss: 0.0023 - mae: 0.0397 - val_loss: 0.0783 - val_mae: 0.2411
Epoch 6/100
411/411 [==============================] - 3s 8ms/step - loss: 0.0022 - mae: 0.0396 - val_loss: 0.0797 - val_mae: 0.2430
Epoch 7/100
411/411 [==============================] - 3s 8ms/step - loss: 0.0022 - mae: 0.0397 - val_loss: 0.0756 - val_mae: 0.2354
Epoch 8/100
411/411 [==============================] - 3s 7ms/step - 